In [1]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    # si usan colab, deben cambiar el token de esta url
    #! mkdir -p ../data
    # los que usan colab deben modificar el token de esta url:
    #! wget -O ../data/properati_2017.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M2/CLASE_07_Pandas_2/Data/resultado-de-encuestas-2017-2018.csv?token=AA4GFHO3EKGWGILBCV4BISC6WR6B4
    
except ModuleNotFoundError:    
    # settings local:
    %run "common/base_setup.py"

Running command `conda list`... ok
pandas=1.0.3 already installed
matplotlib=2.2.2 already installed
bokeh=2.0.0 already installed
seaborn=0.10.0 already installed
ipywidgets=7.5.1 already installed
pytest=5.3.4 already installed
chardet=3.0.4 already installed
plotly=4.6.0 already installed
Running command `conda install --yes chart-studio=1.1.0`... error
    
PackagesNotFoundError: The following packages are not available from current channels:

  - chart-studio=1.1.0

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.





## Carga Inicial del Dataset

In [6]:
import pandas as pd
import common.matchers as match

In [7]:
# low_memorybool, default True
# Internally process the file in chunks, resulting in lower memory use while parsing, 
# but possibly mixed type inference. To ensure no mixed types either set False, 
# or specify the type with the dtype parameter. 
# Note that the entire file is read into a single DataFrame regardless, 
# use the chunksize or iterator parameter to return the data in chunks. (Only valid with C parser).
data = pd.read_csv("./data/properati-fase1.csv", sep = ",", low_memory=False)

## Ambientes

Imputamos rooms faltantes a partir de las colúmnas de título y descripción:

In [8]:
na_rooms_inicio = data["rooms"].isna().sum()
data['rooms'].fillna(data['title'].map(match.match_rooms), inplace=True)
data['rooms'].fillna(data['description'].map(match.match_rooms), inplace=True)
na_rooms_final = data["rooms"].isna().sum()

print("Total registros NA (0): {0}".format(na_rooms_inicio))
print("Total registros NA corregidos: {0}".format(na_rooms_inicio - na_rooms_final))
print("Total registros NA (1): {0}".format(na_rooms_final))

Total registros NA (0): 73830
Total registros NA corregidos: 21299
Total registros NA (1): 52531


## Piso

Imputamos floor faltantes a partir de las colúmnas de título y descripción:

In [9]:
na_floor_inicio = data["floor"].isna().sum()
data['floor'].fillna(data['title'].map(match.match_floor), inplace=True)
data['floor'].fillna(data['description'].map(match.match_floor), inplace=True)
na_floor_final = data["floor"].isna().sum()

print("Total registros NA (0): {0}".format(na_floor_inicio))
print("Total registros NA corregidos: {0}".format(na_floor_inicio - na_floor_final))
print("Total registros NA (1): {0}".format(na_floor_final))

Total registros NA (0): 113298
Total registros NA corregidos: 24275
Total registros NA (1): 89023


## Amenities

Imputamos amenities a partir de la colúmna descripción. Elegimos las siguientes:
 * Pileta
 * Parrilla
 * Quincho
 * Balcón
 * Patio

La forma es imputación es mediante variables dummies. Es decir, para cada amenitie agregamos una columna con el nombre de la misma y la completamos con 1 si el anuncio la indica y 0 en caso contrario.

In [10]:
data["pileta"] = data['description'].map(match.match_amenity_pileta)
print("Total de \"{0}\" imputados: {1}".format("pileta", data["pileta"].sum()))

data["parrilla"] = data['description'].map(match.match_amenity_parrilla)
print("Total de \"{0}\" imputados: {1}".format("parrilla", data["parrilla"].sum()))

data["quincho"] = data['description'].map(match.match_amenity_quincho)
print("Total de \"{0}\" imputados: {1}".format("quincho", data["quincho"].sum()))

data["patio"] = data['description'].map(match.match_amenity_patio)
print("Total de \"{0}\" imputados: {1}".format("patio", data["patio"].sum()))

data["cochera"] = data['description'].map(match.match_amenity_cochera)
print("Total de \"{0}\" imputados: {1}".format("cochera", data["cochera"].sum()))

data["balcon"] = data['description'].map(match.match_amenity_balcon)
print("Total de \"{0}\" imputados: {1}".format("balcon", data["balcon"].sum()))

data['amenities'] = data['pileta'] + data['parrilla'] + data['quincho'] + data['patio'] + data['cochera'] + data['balcon']

Total de "pileta" imputados: 34413
Total de "parrilla" imputados: 38636
Total de "quincho" imputados: 13094
Total de "patio" imputados: 36530
Total de "cochera" imputados: 50472
Total de "balcon" imputados: 43118


In [11]:
data.to_csv('./data/properati-fase2.csv', sep=',', index=False)